### 3.1 Introduction to Lists

All data structures in q are ultimately built from lists: 
- a dictionary is a pair of lists; <br>
- a table is a special dictionary; <br>
- a keyed table is a pair of tables.<br>

All lists are equal but some lists are more equal than others. These are the lists of atoms of homogenous type, called **simple lists** – known in mathematics as vectors. They have optimum storage and performance characteristics.

Lists are not stored as singly-linked lists under the covers. 
- Simple lists occupy contiguous storage and 
- general lists are pointers in contiguous storage.
<br>

- Appending items to the end rather than “cons”-ing to the front.
- Efficient direct item access is available via indexing.
- A general list can hold items of different type without resorting to a union (sum) type.
- It may be instructive to think of q lists as dynamically allocated arrays.

In [1]:
((1; 2; 3); (`1; "2"; 3); 4.4) / general list

1 2 3
(`1;"2";3)
4.4


In [2]:
L1:("a";"b";"c";"d") / simple list
L1

"abcd"


In [3]:
count L1 
count 42  / count of an atom is 1
count `www

4


1


1


In [4]:
first L1
last L1

"a"


"d"


### 3.2 Simple Lists

A list of atoms of a uniform type, called a **simple list**, corresponds to the mathematical notion of a vector. Such lists are treated specially in q. They have a simplified notation, take less storage and compute faster than general lists.
<br>
Whenever q recognizes that the items of a list are homogenous atoms, it dynamically converts to a simple list without asking for permission.

#### 3.2.1 Simple Integer Lists

In [5]:
(100; 200; 300)~100 200 300 / Simple integer list

1b


In [6]:
1 2 3h~(1h; 2h; 3h) / Simple list of shorts
1 2 3h

1b


1 2 3h


#### 3.2.2 Simple Floating Point Lists

In [7]:
1.0 2.0 3.0 / Simple list of floats

1 2 3f


#### 3.2.3 Simple Binary Lists

In [8]:
(0b;1b;0b;1b;1b) / Simple list of booleans

01011b


A simple list of boolean atoms requires the same number of bytes to store as it has atoms. While the simplified notation is suggestive of a bit mask, multiple bits are not compressed to fit inside a single byte. The boolean list above uses 5 bytes of storage.

In [9]:
(0x20;0xa1;0xff) / Simple list of bytes

0x20a1ff


#### 3.2.4 Simple Symbol Lists

In [10]:
(`Life;`the;`Universe;`and;`Everything) / Simple list of symbols
`Life`the`Universe`and`Everything ~ (`Life;`the;`Universe;`and;`Everything) / no whitespaces between symbols!

`Life`the`Universe`and`Everything


1b


#### 3.2.5 Simple char Lists and Strings

In [11]:
("s"; "t"; "r"; "i"; "n"; "g") / Simple list of chars

"string"


A simple list of char is actually called a **string**; however, it does not have the atomic semantics of a string in most languages. Since it is a list of char, not an atom, you cannot ask if two strings of different lengths are equal. You can ask if they are identical (as you can with any two q entities).

In [12]:
"string"~"text"
"text1"="text2"

0b


11110b


#### 3.2.6 Lists of Temporal Data

In [13]:
(2000.01.01; 2001.01.01; 2002.01.01) / Simple list of dates

2000.01.01 2001.01.01 2002.01.01


Specifying a list of mixed temporal types has a different behavior from that of a list of mixed numeric types. In this case, the list takes the type of the first item in the list; other items are widened or narrowed to match.

In [14]:
12:34 01:02:03
01:02:03 12:34

12:34:00 01:02:03


01:02:03 12:34:00


To force the type of a mixed list of temporal values, append a type specifier.

In [15]:
01:02:03 12:34 11:59:59.999u

01:02 12:34 11:59


### 3.3 Empty and Singleton Lists

#### 3.3.1 The General Empty List

In [16]:
() / empty list
count ()
first ()

0


*If you want to force the display of an empty list, use the q utility **-3!**, which converts any q entity into a string suitable for display. Perhaps this utility should be called the “wizard of Oz” operator since it reveals what’s behind the curtain.*

In [17]:
L:()
L
-3!L

"()"


#### 3.3.2 Singleton Lists

A singleton is a list containing a single item. 
<br>
The syntax of a singleton presents a notational conundrum for q. We might expect to write the singleton list containing 42 as (42) but we cannot. The issue arises from q’s use of parentheses both for list demarcation and for the usual mathematical grouping in expressions. The latter leads to the following sequence of arithmetic reductions.
<br>
(40+2) (42) 42
<br>
This forces (42) to be the atom 42.
<br>
Unfortunately, there is no way to type a singleton literal. Singletons are created by the function **enlist**, which “boxes” its argument into a list with one item.

In [18]:
L:enlist 42
count L
L

1


,42


In [19]:
L2:enlist 1 2 3
L2
count L2

1 2 3


1


A string with a single character cannot be written syntactically as “a” – this is a character atom. Use enlist. 

In [20]:
"a"
enlist "a"

"a"


,"a"


### 3.4 Indexing

A list is sequenced from left-to-right in the position of its items. The offset of an item from the beginning of the list is called its index. The initial item has index 0, the second item has index 1, etc. **Thus a list of count n has items at indices 0 through n - 1.**

In [21]:
L:100 200 300
L[1]


200


In [22]:
L[1]:42 / assignment via indexing
L


100 42 300


Index assignment into a simple list enforces strict type matching with no type promotion. Otherwise put, when you assign an item into a simple list, the type must match exactly – i.e., a narrower type is not widened.

In [22]:
L[1]:42h

[0;31mtype[0m: [0;31mtype[0m

In [22]:
L[1.0]

[0;31mtype[0m: [0;31mtype[0m

In [23]:
L[count L] / out-of-bound index results in a null value

0N


In [24]:
L[0W]

0N


#### 3.4.4 Empty Index and Null Item

In [25]:
L[] / An omitted index returns the entire list.
L[::] / :: - nil item

100 42 300


100 42 300


In [26]:
-3!L[()] / indexing with an empty list returns empty list


"()"


The type of the nil item ( :: ) does not match any other type in q. Consequently, inclusion of the nil item in a list forces the list to be general.

In [27]:
L:(::; 1 ; 2 ; 3)
type L
-3!L[0]

0h


"::"


q will automatically convert general list to simple list when all items are of the same type, but won't convert back.
One way to avoid this is by placing :: in the list as a guard.
A drawback of this technique is that you must avoid passing :: on to expressions that use the actual data in the list.

In [27]:
L:(1; 2; 3; `a)
L[3]:4
L
L[3]:`a


1 2 3 4


[0;31mtype[0m: [0;31mtype[0m

In [28]:
L:(::; 1 ; 2 ; 3; `a)
L[4]:4
L[4]:`a
L


::
1
2
3
`a


#### 3.4.5 Lists with Expressions

Any valid q expression can occur in list construction.

In [29]:
L1:1 2 3
L2:40 50
(count L1; sum L2)

3 90


You cannot use the abbreviated notation of simple lists with variables.

In [29]:
a:2
b:3
L:a b

[0;31mtype[0m: [0;31mtype[0m

### 3.5 Combining Lists

#### 3.5.1 Joining with ,

In [30]:
L:1 2,3 4
L

1 2 3 4


To ensure that a q entity becomes a list, use either (),x or x,()

In [31]:
L:enlist 5
L1:(),5
L
L1
L~L1

,5


,5


1b


#### 3.5.2 Merging with ^

Another way to combine two lists of the same length is by coalescing them with ^. The result is given by the rule that the right item prevails over the corresponding left item except when the right item is null.

In [32]:
L1:10 0N 30
L2:100 200 0N
L1^L2

100 200 30


In [33]:
L:(1;2;(100;200))
L

1
2
100 200


### 3.7 Nesting

In [34]:
Nstd: (1; (42; "z"); ((2; "z"); (42; `abv)))
Nstd

1
(42;"z")
((2;"z");(42;`abv))


In [35]:
count Nstd
Nstd[0]
Nstd[1]
Nstd[2]

3


1


42
"z"


2  "z" 
42 `abv


In [36]:
count Nstd[1]
Nstd[1][0]
Nstd[1][1]

2


42


"z"


In [37]:
count Nstd[2]
Nstd[2][0]
Nstd[2][1]

2


2
"z"


42
`abv


In [38]:
count Nstd[2][0]
Nstd[2][1][0]
Nstd[2][1][1]

2


42


`abv


In [39]:
L4: enlist 1 2 3 4
count L4
L4[0]

1


1 2 3 4


#### 3.8.2 Indexing at Depth

In [40]:
Nstd[2;1;0]   // think as multi-dimensional matrix
Nstd[2][1][0] // think as array of arrays

42


42


Assignment via index at depth works but assignment does not work with iterated indexing.

In [41]:
Nstd[2;1;0]:43
Nstd[2;1;0]

43


In [-1]:
Nstd[2][1][0]:44

[0;31mparse error[0m: [0;31massign[0m

It is possible to create a (possibly ragged) array of a given number of rows or columns from a flat list using the **reshape operator #** by specifying 0N (missing data) for the number of rows or columns in the left operand.

In [42]:
2 0N#til 11

0 1 2 3 4
5 6 7 8 9 10


In [43]:
0N 3#til 11

0 1 2
3 4 5
6 7 8
9 10


### 3.9 Indexing with Lists

#### 3.9.1 Retrieving Multiple Items

In [44]:
L:100 200 300 400

Retrieve multiple indexes. <br>
The indices can be in any order, or even be duplicated and the corresponding items are retrieved. The shape of the output conforms to the input.

In [45]:
L[0 2]
L[3 2 0 1]
L[3 3]

100 300


400 300 100 200


400 400


#### 3.9.2 Indexing via a Variable

In [46]:
L
I:0 2
L[I]

100 200 300 400


100 300


#### 3.9.3 Indexing with Nested Lists

Observe that in our examples of list indexing, the result of index retrieval has the same shape as the index. If the index is an atom the result is an atom. When the index list was a simple list, the result was a list of the same count.

More generally, we can retrieve via an arbitrary collection of indices. The retrieved list has the same shape as the index list.

In [47]:
I:((3; 0); (2; 2))
L[I]

400 100
300 300


#### 3.9.4 Assignment with List Indexing

Assignment via index extends to indexing via a simple list with the proviso that the index list and value list conform.

In [48]:
L[1 2 3]:2000 3000 4000

Consequently, in the case of a repeated item in the index list (not a swell idea), the right-most assignment prevails.

In [49]:
L[0 1 0 3]:1000 2000 3000 4000
L

3000 2000 3000 4000


You can assign a single value to multiple items in a list by using an atom for the assignment value. This is an example of a **general phenomenon in q in which an atom is extended to conform to a list.**

In [50]:
L:100 200 300 400
L[1 3]:99
L

100 99 300 99


#### 3.9.5 Juxtaposition

Now that we’re familiar with retrieving and assigning via an index list, we introduce a simplified notation that is common in functional programming. It is permissible to leave out the brackets and juxtapose the list and index with separating whitespace (usually just a blank). 

In [51]:
L 1 3

99 99


In [52]:
I:2 1
L I

300 99


#### 3.9.6 Find (?)

The find operator is (an overload of) dyadic ? that **returns the index of the first occurrence of the right operand in the left operand**. Inverse operation to indexing

In [53]:
1001 1002 1003?1002

1


If you try to find an item that is not in the list, the result is an integer equal to the count of the list.

In [54]:
1001 1002 1003?1004 / not found

3


In [55]:
1001 1002 1003?1003 1001 / multiple 

2 0


### 3.10 Elided Indices

#### 3.10.1 Eliding Indices for a Matrix

In [56]:
m:(1 2 3 4; 100 200 300 400; 1000 2000 3000 4000)
m

1    2    3    4   
100  200  300  400 
1000 2000 3000 4000


Eliding an index in any slot is equivalent to specifying all legitimate indices for that slot.

In [57]:
m[1;] /row
m[;3] / column
m[1;]~m[1] / eliding the last index reduces to item indexing at the top level.

100 200 300 400


4 400 4000


1b


#### 3.10.2 Eliding Indices for Deeply Nested Lists

In [58]:
L:((1 2 3;4 5 6 7);(`a`b`c`d;`z`y`x`;`0`1`2);("now";"is";"the"))
L


(1 2 3;4 5 6 7)
(`a`b`c`d;`z`y`x`;`0`1`2)
("now";"is";"the")


skipped index equivalent to "all indexes"

In [59]:
L[1;;] / extract element with index 1

`a`b`c`d
`z`y`x`
`0`1`2


In [60]:
L[;1;] / for all top-level lists extract elements with index 1

4 5 6 7
`z`y`x`
"is"


In [61]:
L[;;1] / for all elements of top-level lists ( second-level lists) extract element with index 1

2 5
`b`y`1
"osh"


In [62]:
L[0 2;;0 1] / for all elements of top-level lists ( second-level lists) with indices 0 and 2 extract elements with indices 0 and 1

(1 2;4 5)
("no";"is";"th")


In general, it is permissible to elide all trailing semicolons arising from elided indices. We consider this bad practice. 

### 3.11 Rectangular Lists and Matrices

#### 3.11.1 Rectangular Lists

**A rectangular list** is a list of lists all having the same count. This does not mean that a rectangular list is necessarily a traditional matrix, since there can be additional levels of nesting.

In [63]:
L:(1 2 3; (10 20; 100 200; 1000 2000))
L

1         2         3        
10   20   100  200  1000 2000


A rectangular list can be transposed with **flip**, meaning that the rows and columns are reflected across the diagonal. When flip is applied to a rectangular list, **it physically transposes the data – i.e., it allocates new storage and copies the original data in column order.**

In [64]:
M:flip L
-3!M
M

"((1;10 20);(2;100 200);(3;1000 2000))"


1 10 20    
2 100 200  
3 1000 2000


#### 3.11.2 Formal Definition of Matrices

Matrices are a special case of rectangular lists and are defined recursively. A matrix of dimension 0 is a scalar. A matrix of dimension 1 is a simple list – i.e., a vector. The count of a vector is usually called its length or dimension in mathematics.
<br>
For n>1, we define a matrix of dimension n as a list of matrices of dimension n - 1 all having the same size. Thus, a matrix of dimension 2 is a list of vectors, all having the same size. If all atoms in a matrix have the same type, we call this the type of the matrix.

Matrices are nested lists stored in row order. When the rows are simple, each occupies contiguous storage. This makes row retrieval very fast. On the other hand, columns must be picked out of the rows, so column operations are slower.

In [65]:
mm:((1 2;3 4;5 6);(10 20;30 40;50 60))
mm

1 2   3 4   5 6  
10 20 30 40 50 60


In [66]:
mm 0 2 1 / 0 2 1 is treated as list of top-level indexes

1 2   3 4   5 6  
                 
10 20 30 40 50 60


In [67]:
((mm 0) 2) 1

6


### 3.12 Useful List Operations

#### 3.12.1 til

The monadic **til** takes a non-negative integer n and returns a list of n consecutive natural numbers starting at 0. It is useful for constructing regular lists of integers.

In [68]:
til 10

0 1 2 3 4 5 6 7 8 9


In [69]:
-5+4*til 3

-5 -1 3


#### 3.12.2 distinct

The monadic function **distinct** returns the unique items in its list argument, in order of first occurrence.

In [70]:
distinct 1 2 3 2 3 4 6 4 3 5 6

1 2 3 4 6 5


#### 3.12.3 where

The basic form of **where** returns the indices of 1b in a boolean list – i.e., it reports where the ones are.

In [71]:
where 101010b

0 2 4


In [72]:
L:10 20 30 40 50
L[where L>20]

30 40 50


In [73]:
L[where L>20]:555
L

10 20 555 555 555


#### 3.12.4 group

The monadic function **group** takes a list and returns a dictionary in which each distinct item of the argument is mapped to the indices of its occurrences, in order of occurrence.

In [74]:
group "i miss mississippi"

i| 0 3 8 11 14 17
 | 1 6
m| 2 7
s| 4 5 9 10 12 13
p| 15 16


In [75]:
group 1 2 3 2 3 4 6 4 3 5 6

1| ,0
2| 1 3
3| 2 4 8
4| 5 7
6| 6 10
5| ,9
